# Data Aggregation and Group Operations

In [1]:
import numpy as np
import pandas as pd

## GroupBy Mechanics

In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,1.265206,0.756538
1,a,two,-0.493620,0.761695
2,b,one,0.113637,1.309112
3,b,two,0.754402,0.219837
4,a,one,-0.015133,0.977518


* Islemi vermezsen asagidaki cikar o yuzden islem tanimlamalisin...

In [4]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [5]:
grouped.mean()

key1
a    0.252151
b    0.434020
Name: data1, dtype: float64

* Hem key1 e hem de key2 ye gore gruplar

In [6]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.625037
      two    -0.493620
b     one     0.113637
      two     0.754402
Name: data1, dtype: float64

In [7]:
means.unstack()

key2,one,two
key1,,
a,0.625037,-0.493620
b,0.113637,0.754402


In [9]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005   -0.493620
            2006    0.113637
Ohio        2005    1.009804
            2006   -0.015133
Name: data1, dtype: float64

In [10]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.252151,0.831917
b,0.434020,0.764474


In [11]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.625037  0.867028
     two  -0.493620  0.761695
b    one   0.113637  1.309112
     two   0.754402  0.219837

In [12]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterating Over Groups

In [13]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  1.265206  0.756538
1    a  two -0.493620  0.761695
4    a  one -0.015133  0.977518
b
  key1 key2     data1     data2
2    b  one  0.113637  1.309112
3    b  two  0.754402  0.219837


### Selecting a Column or Subset of Columns

df.groupby('key1')['data1']
df.groupby('key1')[['data2']]

df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])

In [14]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.867028
     two   0.761695
b    one   1.309112
     two   0.219837

### Grouping with Dicts and Series

In [15]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,-0.271298,0.723333,0.152757,1.486442,0.803156
Steve,0.006092,0.774269,-0.270747,2.300168,0.045427
Wes,0.367759,NaN,NaN,-1.229019,0.086459
Jim,-1.001499,0.777702,1.194400,-0.250153,-0.794877
Travis,0.526107,-0.620540,-0.654698,1.422388,-0.577468


In [16]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [17]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,1.639199,1.255191
Steve,2.029421,0.825788
Wes,-1.229019,0.454218
Jim,0.944247,-1.018674
Travis,0.767690,-0.671901


In [18]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [19]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### Grouping with Functions

In [20]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.905038,1.501035,1.347156,0.007271,0.094738
5,0.006092,0.774269,-0.270747,2.300168,0.045427
6,0.526107,-0.620540,-0.654698,1.422388,-0.577468


## Data Aggregation

In [21]:
df
grouped = df.groupby('key1')

In [22]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.758826,0.220980
b,0.640764,1.089275


In [23]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.252151  0.909367 -0.493620 -0.254376 -0.015133  0.625037   
b      2.0  0.434020  0.453089  0.113637  0.273828  0.434020  0.594211   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.265206   3.0  0.831917  0.126121  0.756538  0.759116  0.761695   
b     0.754402   2.0  0.764474  0.770234  0.219837  0.492156  0.764474   

                          
           75%       max  
key1                      
a     0.869606  0.977518  
b     1.036793  1.309112

### Column-Wise and Multiple Function Application

In [24]:
tips = pd.read_csv('examples/tips.csv')
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [26]:
grouped = tips.groupby(['day', 'smoker'])

In [28]:
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')  #.mean() desen de calisirdi...

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [29]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [30]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)]) #Isim veriyor usttekinden farkli olarak...

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [31]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

## Pivot Tables

In [32]:
tips.pivot_table(index=['day', 'smoker'])   #Yazanlarin hepsi ortalama...

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [35]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',    #Simdi len gostericek mean yerine... margin all satirinin gorunmesini saglar.
                 aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0